# Feature Based con K=6

#### Se debe ejecutar primero FeatureBased con K=5

##### -------------------------------------------------------------------------

### Inputs files: 

#### filesnotificaciones_clima_7_dias_COMBINADO_SEMANAL.csv
#### Poblacion_Paraguay.csv

##### -------------------------------------------------------------------------

### Output files: 

#### Scores1_FeatureBasedK6.csv
#### ClusterFB1_2009al2013K6.csv

##### -------------------------------------------------------------------------

In [1]:
from sklearn.metrics import davies_bouldin_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tsfeature as tsf
import entropy as ent

my_path = os.path.abspath('')
my_path = my_path.split('\\')
my_path_py = "\\".join(my_path[:-1])

In [3]:
df= pd.read_csv(my_path_py+'\\DatosRaw\\notificaciones_clima_7_dias_COMBINADO_SEMANAL.csv', sep=';')
poblacion= pd.read_csv(my_path_py+'\\DatosRaw\\Poblacion_Paraguay.csv', sep=',')

In [4]:
poblacion['#'] = poblacion['#'].str.upper()
poblacion = poblacion.head(268)
poblacion.rename(columns = {'#':'Ciudad'}, inplace = True)

In [5]:
print("La cantidad de ciudades con casos es:", df['distrito_notif'].nunique())
print("La cantidad de ciudades con datos de población:", poblacion['Ciudad'].nunique())

La cantidad de ciudades con casos es: 238
La cantidad de ciudades con datos de población: 266


In [6]:
S1 = set(list(dict.fromkeys(df['distrito_notif'].tolist())))
S2 = set(list(dict.fromkeys(poblacion['Ciudad'].tolist())))
ciudades = S1.intersection(S2)
df

,anio,semana,distrito_notif,tmax,tmin,tmed,td,pres_est,pres_nm,prcp,hr,helio,nub,vmax_d,vmax_f,vmed,id_estacion,casos
0,2009,2,ASUNCION,29.7,20.5,23.9,16.85,1001.05,1010.55,2,68,6.6,4.5,??,??,??,86218,2
1,2009,2,FERNANDO DE LA MORA,38,24.6,29.2,22.1,998.3,1007.6,0,68,11.1,5,??,??,??,86218,1
2,2009,2,PEDRO JUAN CABALLERO,27.7,21.3,23.1,20.9,947,1008.3,32.6,88.5,1.85,6.5,??,??,??,86097,2
3,2009,2,YBY YAU,27.4,20.65,22.65,20.475,947.1,1008.5,16.775,88.5,2.875,6.5,??,??,??,86097,4
4,2009,2,YPANE,29,20.2,23.3,16,1001.6,1011.2,0,66,7.5,6,??,??,??,86218,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8482,2013,52,VILLA HAYES,35.46666667,24.48333333,29.86666667,21.86666667,998.2666667,1007.683333,2.133333333,65.16666667,10.61666667,4,14.66666667,2.126666667,1.308333333,86218,6
8483,2013,52,VILLETA,34.53333333,23.6,28.7,21.73333333,996.6666667,1006.1,0.1,67.66666667,7.866666667,3.333333333,14.33333333,1.573333333,0.786666667,86218,3
8484,2013,52,YAGUARON,35,22.8,29.7,23.4,995.3,1008.4,0,70,10,1,0,0,0,86221,1
8485,2013,52,YPACARAI,36,24,29.2,21.4,994.7,1007.9,0,65,10.7,1,18,1.11,0.28,86221,2


In [7]:
df = df[['anio','distrito_notif', 'semana', 'casos']]


In [8]:
df.rename(columns = {'anio':'Año', 'distrito_notif':'Distrito'}, inplace = True)
df = df[df['Distrito'].isin(ciudades)]
listaDistrito = df['Distrito'].tolist()
listaDistrito = list(dict.fromkeys(listaDistrito))

In [9]:
features= pd.read_csv('FB2_2009al2013.csv', sep=',')
features

,Distrito,Mean,Var,ACF1,Trend,Linearity,Curvature,Season,Peak,Trough,Entropy,Lumpiness,Spikiness,Lshift,Vchange,Fspots,Cpoints,KlScore,ChangeIdx
0,ASUNCION,6.904494,5.239362e+01,8.135193e-01,0.922999,0.003082,-0.863262,0.025780,1.000000e+00,-0.010495,5.978370e-01,1.772715e-04,2.497263e-09,1.086957e-02,5.250998e-05,172.0,6.0,1.000000e-10,1.000000e-10
1,FERNANDO DE LA MORA,44.781615,4.841265e+03,9.371226e-01,0.958335,0.007501,0.391511,0.046909,1.000000e-07,-0.010903,1.698908e-01,7.906015e-05,5.201407e-10,1.813156e-01,1.006363e-02,49.0,10.0,1.000000e-10,1.000000e-10
2,PEDRO JUAN CABALLERO,44.819130,1.042302e+04,9.285754e-01,0.962853,0.004186,-0.361289,0.010942,1.000000e-07,-0.004989,1.317796e-01,3.606045e-04,5.397936e-10,6.596210e-02,3.044544e-03,58.0,2.0,1.000000e-10,1.000000e-10
3,YBY YAU,18.306891,8.273701e+02,7.051710e-01,0.879186,0.003470,-0.677443,0.049205,1.000000e+00,-0.018949,5.381565e-01,3.756719e-04,3.576345e-09,5.000000e-02,2.083333e-03,120.0,3.0,1.000000e-10,1.000000e-10
4,YPANE,59.676847,1.191727e+04,9.103049e-01,0.940326,0.005569,-0.154082,0.019642,1.000000e-07,-0.005024,1.524750e-01,8.255672e-05,1.405710e-09,1.234592e-01,7.082626e-03,84.0,4.0,1.000000e-10,1.000000e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ISLA UMBU,33.333333,1.000000e-07,1.000000e-08,0.984279,0.000052,-0.016121,0.015653,1.000000e-07,-0.001953,1.000000e-08,1.000000e-07,3.203435e-11,1.000000e-07,1.000000e-07,193.0,0.0,1.000000e-10,1.000000e-10
196,VILLA OLIVA,27.129680,1.000000e-07,1.000000e-08,0.984279,0.000052,-0.016121,0.015653,1.000000e-07,-0.001953,1.000000e-08,1.000000e-07,3.203435e-11,1.000000e-07,1.000000e-07,193.0,0.0,1.000000e-10,1.000000e-10
197,CARMELO PERALTA,23.540490,1.000000e-07,1.000000e-08,0.899220,0.000466,-0.144232,0.020585,1.000000e-07,-0.007148,1.000000e-08,2.266590e-03,5.186532e-09,1.000000e-07,1.000000e-07,191.0,1.0,1.000000e-10,1.000000e-10
198,DESMOCHADOS,56.497175,1.000000e-07,1.000000e-08,0.984279,0.000052,-0.016121,0.015653,1.000000e-07,-0.001953,1.000000e-08,1.000000e-07,3.203435e-11,1.000000e-07,1.000000e-07,193.0,0.0,1.000000e-10,1.000000e-10


In [10]:
n= features.shape[0]
n

200

# Funciones de Distancias

In [11]:
import math
from math import sqrt, log, floor
from sklearn.metrics import mean_squared_error
from statistics import mean
from fastdtw import fastdtw
from scipy import stats
from scipy.spatial.distance import pdist

#Euclidean
def euclidean(x, y):
    r=np.linalg.norm(x-y)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Fast Dynamic time warping
def fast_DTW(x, y):
    r, _ = fastdtw(x, y, dist=euclidean)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Spearman
def scorr(x, y):
    r = stats.spearmanr(x, y)[0]
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

#RMSE
def rmse(x, y):
    r=sqrt(mean_squared_error(x,y))
    if math.isnan(r):
        r=1
    #print(r)
    return r

def lcs(a, b):  
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    x, y = len(a), len(b)
    result = lengths[x][y]
    return result

def discretise(x):
    return int(x * 10)

def multidim_lcs(a, b):
    a = a.applymap(discretise)
    b = b.applymap(discretise)
    rows, dims = a.shape
    lcss = [lcs(a[i+2], b[i+2]) for i in range(dims)]
    return 1 - sum(lcss) / (rows * dims)

#Correlation
def corr(x, y):
    r=np.dot(x-mean(x),y-mean(y))/((np.linalg.norm(x-mean(x)))*(np.linalg.norm(y-mean(y))))
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

In [12]:
from sklearn.preprocessing import LabelEncoder
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score

DIAMETER_METHODS = ['mean_cluster', 'farthest']
CLUSTER_DISTANCE_METHODS = ['nearest', 'farthest']

def inter_cluster_distances(labels, distances, method='nearest'):
    """Calculates the distances between the two nearest points of each cluster.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: `nearest` for the distances between the two nearest points in each cluster, or `farthest`
    """
    if method not in CLUSTER_DISTANCE_METHODS:
        raise ValueError(
            'method must be one of {}'.format(CLUSTER_DISTANCE_METHODS))

    if method == 'nearest':
        return __cluster_distances_by_points(labels, distances)
    elif method == 'farthest':
        return __cluster_distances_by_points(labels, distances, farthest=True)


def __cluster_distances_by_points(labels, distances, farthest=False):
    n_unique_labels = len(np.unique(labels))
    cluster_distances = np.full((n_unique_labels, n_unique_labels),
                                float('inf') if not farthest else 0)

    np.fill_diagonal(cluster_distances, 0)

    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i, len(labels)):
            if labels[i] != labels[ii] and (
                (not farthest and
                 distances[i, ii] < cluster_distances[labels[i], labels[ii]])
                    or
                (farthest and
                 distances[i, ii] > cluster_distances[labels[i], labels[ii]])):
                cluster_distances[labels[i], labels[ii]] = cluster_distances[
                    labels[ii], labels[i]] = distances[i, ii]
    return cluster_distances


def diameter(labels, distances, method='farthest'):
    """Calculates cluster diameters
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: either `mean_cluster` for the mean distance between all elements in each cluster, or `farthest` for the distance between the two points furthest from each other
    """
    if method not in DIAMETER_METHODS:
        raise ValueError('method must be one of {}'.format(DIAMETER_METHODS))

    n_clusters = len(np.unique(labels))
    diameters = np.zeros(n_clusters)

    if method == 'mean_cluster':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii]:
                    diameters[labels[i]] += distances[i, ii]

        for i in range(len(diameters)):
            diameters[i] /= sum(labels == i)

    elif method == 'farthest':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii] and distances[i, ii] > diameters[
                        labels[i]]:
                    diameters[labels[i]] = distances[i, ii]
    return diameters

def dunn(labels, distances, diameter_method='farthest',
         cdist_method='nearest'):
    """
    Dunn index for cluster validation (larger is better).
    
    .. math:: D = \\min_{i = 1 \\ldots n_c; j = i + 1\ldots n_c} \\left\\lbrace \\frac{d \\left( c_i,c_j \\right)}{\\max_{k = 1 \\ldots n_c} \\left(diam \\left(c_k \\right) \\right)} \\right\\rbrace
    
    where :math:`d(c_i,c_j)` represents the distance between
    clusters :math:`c_i` and :math:`c_j`, and :math:`diam(c_k)` is the diameter of cluster :math:`c_k`.
    Inter-cluster distance can be defined in many ways, such as the distance between cluster centroids or between their closest elements. Cluster diameter can be defined as the mean distance between all elements in the cluster, between all elements to the cluster centroid, or as the distance between the two furthest elements.
    The higher the value of the resulting Dunn index, the better the clustering
    result is considered, since higher values indicate that clusters are
    compact (small :math:`diam(c_k)`) and far apart (large :math:`d \\left( c_i,c_j \\right)`).
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param diameter_method: see :py:function:`diameter` `method` parameter
    :param cdist_method: see :py:function:`diameter` `method` parameter
    
    .. [Kovacs2005] Kovács, F., Legány, C., & Babos, A. (2005). Cluster validity measurement techniques. 6th International Symposium of Hungarian Researchers on Computational Intelligence.
    """

    labels = LabelEncoder().fit(labels).transform(labels)
    
    

    ic_distances = inter_cluster_distances(labels, distances, cdist_method)
    #print("IC",ic_distances)
    if len(ic_distances[ic_distances.nonzero()])==0:
        min_distance = 0
    else:
        min_distance = min(ic_distances[ic_distances.nonzero()])
    max_diameter = max(diameter(labels, distances, diameter_method))
    
    

    return min_distance / max_diameter

In [13]:
features = features.replace('nan', np.nan).fillna(0.00001)

In [14]:
features = features.drop('Distrito', axis=1)

# DEFINICIÓN DEL NÚMERO DE CLUSTERS

In [15]:
import matplotlib.pyplot as plt
k=6

### Matriz Euclidiana

In [16]:
#Euclidean
f_euclidean_dist = np.zeros((n,n))
for i in range(0,n):
    #print("i",i)
    for j in range(1,n):
         f_euclidean_dist[i,j] = euclidean(features.iloc[i].values.flatten(), features.iloc[j].values.flatten())

In [17]:
np.savetxt('MatrizEuclidiana_FB_K6.csv', f_euclidean_dist, delimiter=",")

### Matriz Correlación Pearson

In [18]:
#Corr
corr_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
            corr_dist[i,j] = corr(features.iloc[i].values.flatten(), features.iloc[j].values.flatten())

In [19]:
np.savetxt('MatrizPearson_FB_K6.csv', corr_dist, delimiter=",")

### Matriz Correlación Spearman 

In [20]:
#scorr
f_scorr_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
        f_scorr_dist[i,j] = scorr(features.iloc[i].values.flatten(), features.iloc[j].values.flatten())

In [21]:
np.savetxt('MatrizSpearman_FB_K6.csv', f_scorr_dist, delimiter=",")

### Matriz DTW

In [22]:
#DTW
f_dtw_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
        f_dtw_dist[i,j] = fast_DTW(features.iloc[i].values.flatten(), features.iloc[j].values.flatten())

In [23]:
np.savetxt('MatrizDTW_FB_K6.csv', f_dtw_dist, delimiter=",")

# CLUSTERING

In [24]:
#Experimentos HAC
HAC_EUCLIDEAN=[]
HAC_CORRELATION=[]
HAC_SPEARMAN=[]
HAC_DTW=[]

HAC_EUCLIDEAN_CHZ=[]
HAC_CORRELATION_CHZ=[]
HAC_SPEARMAN_CHZ=[]
HAC_DTW_CHZ=[]

HAC_EUCLIDEAN_DUNN=[]
HAC_CORRELATION_DUNN=[]
HAC_SPEARMAN_DUNN=[]
HAC_DTW_DUNN=[]

HAC_EUCLIDEAN_DAVID=[]
HAC_CORRELATION_DAVID=[]
HAC_SPEARMAN_DAVID=[]
HAC_DTW_DAVID=[]

HAC_euc = AgglomerativeClustering(n_clusters=k).fit_predict(f_euclidean_dist)
print("HAC + euclidian distance: ")
sil = silhouette_score(f_euclidean_dist, HAC_euc)
print("SC: ", sil)
chz = calinski_harabasz_score(f_euclidean_dist, HAC_euc)
print("CHZ: ", chz)
dunn_= dunn(HAC_euc, f_euclidean_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_= davies_bouldin_score(f_euclidean_dist, HAC_euc)
print("DAV-BOUD: ", david_)
HAC_EUCLIDEAN.append(sil)
HAC_EUCLIDEAN_CHZ.append(chz)
HAC_EUCLIDEAN_DUNN.append(dunn_)
HAC_EUCLIDEAN_DAVID.append(david_)

HAC_corr = AgglomerativeClustering(n_clusters=k).fit_predict(corr_dist)
print("HAC + corr distance: ")
sil = silhouette_score(corr_dist, HAC_corr)
print("SC: ", sil)
chz = calinski_harabasz_score(corr_dist, HAC_corr)
print("CHZ: ", chz)
dunn_ = dunn(HAC_corr, corr_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_= davies_bouldin_score(corr_dist, HAC_corr)
print("DAV-BOUD: ", david_)
HAC_CORRELATION.append(sil)
HAC_CORRELATION_CHZ.append(chz)
HAC_CORRELATION_DUNN.append(dunn_)
HAC_CORRELATION_DAVID.append(david_)

HAC_scorr = AgglomerativeClustering(n_clusters=k).fit_predict(f_scorr_dist)
print("HAC + scorr distance: ")
sil = silhouette_score(f_scorr_dist, HAC_scorr)
print("SC: ", sil)
chz = calinski_harabasz_score(f_scorr_dist, HAC_scorr)
print("CHZ: ", chz)
dunn_ = dunn(HAC_scorr, f_scorr_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_= davies_bouldin_score(f_scorr_dist, HAC_scorr)
print("DAV-BOUD: ", david_)
HAC_SPEARMAN.append(sil)
HAC_SPEARMAN_CHZ.append(chz)
HAC_SPEARMAN_DUNN.append(dunn_)
HAC_SPEARMAN_DAVID.append(david_)

HAC_dtw = AgglomerativeClustering(n_clusters=k).fit_predict(f_dtw_dist)
print("HAC + dtw distance: ")
sil = silhouette_score(f_dtw_dist, HAC_dtw)
print("SC: ", sil)
chz = calinski_harabasz_score(f_dtw_dist, HAC_dtw)
print("CHZ: ", chz)
dunn_ = dunn(HAC_dtw, f_dtw_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_= davies_bouldin_score(f_dtw_dist, HAC_dtw)
print("DAV-BOUD: ", david_)
HAC_DTW.append(sil)
HAC_DTW_CHZ.append(chz)
HAC_DTW_DUNN.append(dunn_)
HAC_DTW_DAVID.append(david_)
print("-----------------------")

HAC + euclidian distance: 
SC:  0.8369767125735053
CHZ:  2742.2766327763857
DUNN:  1.2758936028566583
DAV-BOUD:  0.3951131390544735
HAC + corr distance: 
SC:  0.6146927927802859
CHZ:  1807.2673708790514
DUNN:  5.039794930806593
DAV-BOUD:  0.4562566129799433
HAC + scorr distance: 
SC:  0.57198661536353
CHZ:  386.43085920210643
DUNN:  0.0
DAV-BOUD:  0.7158918514778642
HAC + dtw distance: 
SC:  0.8270216138722725
CHZ:  2699.3107187795003
DUNN:  0.38075437571440324
DAV-BOUD:  0.34879191461132203
-----------------------


In [25]:
KM_EUCLIDEAN=[]
KM_CORRELATION=[]
KM_SPEARMAN=[]
KM_DTW=[]

KM_EUCLIDEAN_CHZ=[]
KM_CORRELATION_CHZ=[]
KM_SPEARMAN_CHZ=[]
KM_DTW_CHZ=[]

KM_EUCLIDEAN_DUNN=[]
KM_CORRELATION_DUNN=[]
KM_SPEARMAN_DUNN=[]
KM_DTW_DUNN=[]

KM_EUCLIDEAN_DAVID=[]
KM_CORRELATION_DAVID=[]
KM_SPEARMAN_DAVID=[]
KM_DTW_DAVID=[]
#Experimentos K-Means
km_euc = KMeans(n_clusters=k).fit_predict(f_euclidean_dist)
print("KM + euclidian distance: ")
sil = silhouette_score(f_euclidean_dist, km_euc)
print("SC: ", sil)
chz = calinski_harabasz_score(f_euclidean_dist, km_euc)
print("CHZ: ", chz)
dunn_ = dunn(km_euc, f_euclidean_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_euclidean_dist, km_euc)
print("DAV-BOULD: ", david_)
KM_EUCLIDEAN.append(sil)
KM_EUCLIDEAN_CHZ.append(chz)
KM_EUCLIDEAN_DUNN.append(dunn_)
KM_EUCLIDEAN_DAVID.append(david_)

km_corr = KMeans(n_clusters=k).fit_predict(corr_dist)
print("KM + corr distance: ")
sil = silhouette_score(corr_dist, km_corr)
print("SC: ", sil)
chz = calinski_harabasz_score(corr_dist, km_corr)
print("CHZ: ", chz)
dunn_ = dunn(km_corr, corr_dist, 'farthest', 'farthest') 
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(corr_dist, km_corr)
print("DAV-BOULD: ", david_)
KM_CORRELATION.append(sil)
KM_CORRELATION_CHZ.append(chz)
KM_CORRELATION_DUNN.append(dunn_)
KM_CORRELATION_DAVID.append(david_)

km_scorr = KMeans(n_clusters=k).fit_predict(f_scorr_dist)
print("KM + scorr distance: ")
sil = silhouette_score(f_scorr_dist, km_scorr)
print("SC: ", sil)
chz = calinski_harabasz_score(f_scorr_dist, km_scorr)
print("CHZ: ", chz)
dunn_ = dunn(km_scorr, f_scorr_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_scorr_dist, km_scorr)
print("DAV-BOULD: ", david_)
KM_SPEARMAN.append(sil)
KM_SPEARMAN_CHZ.append(chz)
KM_SPEARMAN_DUNN.append(dunn_)
KM_SPEARMAN_DAVID.append(david_)

km_dtw = KMeans(n_clusters=k).fit_predict(f_dtw_dist)
print("KM + dtw distance: ")
sil = silhouette_score(f_dtw_dist, km_dtw)
print("SC: ", sil)
chz = calinski_harabasz_score(f_dtw_dist, km_dtw)
print("CHZ: ", chz)
dunn_ = dunn(km_dtw, f_dtw_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_dtw_dist, km_dtw)
print("DAV-BOULD: ", david_)
KM_DTW.append(sil)
KM_DTW_CHZ.append(chz)
KM_DTW_DUNN.append(dunn_)
KM_DTW_DAVID.append(david_)
print("-----------------------")

KM + euclidian distance: 
SC:  0.8630194086688542
CHZ:  3238.982031182185
DUNN:  1.4746546750805811
DAV-BOULD:  0.3311495868455252
KM + corr distance: 
SC:  0.6465186244464638
CHZ:  2178.141310666236
DUNN:  5.068902722050898
DAV-BOULD:  0.4590616704078662
KM + scorr distance: 
SC:  0.6019297578764374
CHZ:  396.53642220947626
DUNN:  0.0
DAV-BOULD:  0.694827369793503
KM + dtw distance: 
SC:  0.8610517516273068
CHZ:  3164.7370663685992
DUNN:  1.4930817578245092
DAV-BOULD:  0.32984662213689697
-----------------------


In [82]:
DBSCAN_EUCLIDEAN=[]
DBSCAN_CORRELATION=[]
DBSCAN_SPEARMAN=[]
DBSCAN_DTW=[]

DBSCAN_EUCLIDEAN_CHZ=[]
DBSCAN_CORRELATION_CHZ=[]
DBSCAN_SPEARMAN_CHZ=[]
DBSCAN_DTW_CHZ=[]

DBSCAN_EUCLIDEAN_DUNN=[]
DBSCAN_CORRELATION_DUNN=[]
DBSCAN_SPEARMAN_DUNN=[]
DBSCAN_DTW_DUNN=[]

DBSCAN_EUCLIDEAN_DAVID=[]
DBSCAN_CORRELATION_DAVID=[]
DBSCAN_SPEARMAN_DAVID=[]
DBSCAN_DTW_DAVID=[]

#CON EUCLIDEAN
DB_euc = DBSCAN(eps=500, min_samples=4).fit_predict(f_euclidean_dist)
print("DBSCAN + euclidian distance: ")
sil =  silhouette_score(f_euclidean_dist, DB_euc)
print("SC: ", sil)
CHZ_ = calinski_harabasz_score(f_euclidean_dist, DB_euc) 
print("CHZ: ", CHZ_)
dunn_ = dunn(DB_euc, f_euclidean_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_euclidean_dist, DB_euc)
print("DAV-BOULD: ", david_)
DBSCAN_EUCLIDEAN.append(sil)
DBSCAN_EUCLIDEAN_CHZ.append(CHZ_)
DBSCAN_EUCLIDEAN_DUNN.append(dunn_)
DBSCAN_EUCLIDEAN_DAVID.append(david_)

#CON CORRELATION
DB_corr = DBSCAN(eps=0.4, min_samples=4).fit_predict(corr_dist)
print("DBSCAN + corr distance: ")
sil = silhouette_score(corr_dist, DB_corr)
print("SC: ", sil)
CHZ_ = calinski_harabasz_score(corr_dist, DB_corr)
print("CHZ: ", CHZ_)
dunn_ = dunn(DB_corr, corr_dist, 'farthest', 'farthest')
print("DUNN: ",)
david_ = davies_bouldin_score(corr_dist, DB_corr)
print("DAV-BOULD: ", david_)
DBSCAN_CORRELATION.append(sil)
DBSCAN_CORRELATION_CHZ.append(CHZ_)
DBSCAN_CORRELATION_DUNN.append(dunn_)
DBSCAN_CORRELATION_DAVID.append(david_)

#CON SPEARMAN
DB_scorr = DBSCAN(eps=0.4, min_samples=4).fit_predict(f_scorr_dist)
print("DBSCAN + scorr distance: ")
sil = silhouette_score(f_scorr_dist, DB_scorr)
print("SC: ", sil)
CHZ_ = calinski_harabasz_score(f_scorr_dist, DB_scorr)
print("CHZ: ", CHZ_)
dunn_ = dunn(DB_scorr, f_scorr_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_scorr_dist, DB_scorr)
print("DAV-BOULD: ", david_)
DBSCAN_SPEARMAN.append(sil)
DBSCAN_SPEARMAN_CHZ.append(CHZ_)
DBSCAN_SPEARMAN_DUNN.append(dunn_)
DBSCAN_SPEARMAN_DAVID.append(david_)

#CON D TIME WARPING
DB_dtw = DBSCAN(eps=500, min_samples=4).fit_predict(f_dtw_dist)
print("DBSCAN + dtw distance: ")
sil = silhouette_score(f_dtw_dist, DB_dtw)
print("SC: ", sil)
CHZ_ = calinski_harabasz_score(f_dtw_dist, DB_dtw)
print("CHZ: ", CHZ_)
dunn_ = dunn(DB_dtw, f_dtw_dist, 'farthest', 'farthest')
print("DUNN: ", dunn_)
david_ = davies_bouldin_score(f_dtw_dist, DB_dtw)
print("DAV-BOULD: ", david_)
DBSCAN_DTW.append(sil)
DBSCAN_DTW_CHZ.append(CHZ_)
DBSCAN_DTW_DUNN.append(dunn_)
DBSCAN_DTW_DAVID.append(david_)

DBSCAN + euclidian distance: 
SC:  0.38137143796771467
CHZ:  14.568195181109074
DUNN:  0.001842577705547823
DAV-BOULD:  1.1297310737849198
DBSCAN + corr distance: 
SC:  0.32026800441110714
CHZ:  426.2008139229266
DUNN: 
DAV-BOULD:  0.5146606588804018
DBSCAN + scorr distance: 
SC:  0.5924323213851097
CHZ:  158.97621012769451
DUNN:  0.0
DAV-BOULD:  1.0407991393298126
DBSCAN + dtw distance: 
SC:  0.5175923960115791
CHZ:  31.070024281328855
DUNN:  1.0007970449603274
DAV-BOULD:  1.2133298906377463


In [83]:
KM1 = KM_EUCLIDEAN + KM_CORRELATION + KM_SPEARMAN + KM_DTW
KM2 = KM_EUCLIDEAN_CHZ + KM_CORRELATION_CHZ + KM_SPEARMAN_CHZ + KM_DTW_CHZ
KM3 = KM_EUCLIDEAN_DUNN + KM_CORRELATION_DUNN + KM_SPEARMAN_DUNN + KM_DTW_DUNN
KM4 = KM_EUCLIDEAN_DAVID + KM_CORRELATION_DAVID + KM_SPEARMAN_DAVID + KM_DTW_DAVID
HAC1 = HAC_EUCLIDEAN + HAC_CORRELATION + HAC_SPEARMAN + HAC_DTW
HAC2 = HAC_EUCLIDEAN_CHZ + HAC_CORRELATION_CHZ + HAC_SPEARMAN_CHZ + HAC_DTW_CHZ
HAC3 = HAC_EUCLIDEAN_DUNN + HAC_CORRELATION_DUNN + HAC_SPEARMAN_DUNN + HAC_DTW_DUNN
HAC4 = HAC_EUCLIDEAN_DAVID + HAC_CORRELATION_DAVID + HAC_SPEARMAN_DAVID + HAC_DTW_DAVID
DBS1 = DBSCAN_EUCLIDEAN + DBSCAN_CORRELATION + DBSCAN_SPEARMAN + DBSCAN_DTW
DBS2 = DBSCAN_EUCLIDEAN_CHZ + DBSCAN_CORRELATION_CHZ + DBSCAN_SPEARMAN_CHZ + DBSCAN_DTW_CHZ
DBS3 = DBSCAN_EUCLIDEAN_DUNN + DBSCAN_CORRELATION_DUNN + DBSCAN_SPEARMAN_DUNN + DBSCAN_DTW_DUNN
DBS4 = DBSCAN_EUCLIDEAN_DAVID + DBSCAN_CORRELATION_DAVID + DBSCAN_SPEARMAN_DAVID + DBSCAN_DTW_DAVID

In [84]:
sil_scores = pd.DataFrame()
sil_scores['METRICA'] = ['Euclidean Distance', 'Pearson Correlation', 'Spearman Correlation', 'Dynamic Time Warping'] 
sil_scores['SIL - HAC'] = np.array(HAC1)
sil_scores['SIL - KM'] = np.array(KM1)
sil_scores['SIL - DB'] = np.array(DBS1)
sil_scores['CHZ - HAC'] = np.array(HAC2)
sil_scores['CHZ - KM'] = np.array(KM2)
sil_scores['CHZ - DB'] = np.array(DBS2)
sil_scores['DUNN - HAC'] = np.array(HAC3)
sil_scores['DUNN - KM'] = np.array(KM3)
sil_scores['DUNN - DB'] = np.array(DBS3)
sil_scores['DAVID - HAC'] = np.array(HAC4)
sil_scores['DAVID - KM'] = np.array(KM4)
sil_scores['DAVID - DB'] = np.array(DBS4)

In [85]:
sil_scores.to_csv('Scores1_FeatureBasedK6.csv', index = False)

In [86]:
aux2 = pd.DataFrame()
aux2['Distrito'] = listaDistrito
aux2['Cluster KM'] = km_euc
aux2['Cluster HAC '] = HAC_euc
aux2['Cluster DB SP'] = DB_corr
aux2.to_csv('ClusterFB1_2009al2013K6.csv', index = False)

In [90]:
np.unique(DB_euc)

array([-1,  0,  1,  2], dtype=int64)

In [88]:
np.unique(HAC_euc)

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [34]:
np.unique(km_euc)

array([0, 1, 2, 3, 4, 5])